In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr  6 18:24:57 2022

@author: Aston
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib
import netCDF4
import scipy.stats
import warnings
from matplotlib.ticker import AutoMinorLocator

# MAtplotlib resources.
plt.rcParams.update({
    'font.family':'sans-serif',
    'font.sans-serif':['Liberation Sans'],
    'font.size':'12',
    })

#1. Open database and import variables.
# Open CSV file and import variables.
csv_import = pd.read_csv('G:/WRF_Schemes/Kennneth/Idai_wnsp_pres.csv', keep_default_na=True, delimiter=',',
 header=None, names=['Wind_I','Pressure_I'])
wind_i = csv_import['Wind_I']
wind_i = wind_i.values
pres_i = csv_import['Pressure_I']
pres_i = pres_i.values

csv_import = pd.read_csv('G:/WRF_Schemes/Kennneth/Kenneth_wspd_pres.csv',keep_default_na=True, delimiter=','
, header=None,names=['Wind_K','Pressure_K'])
wind_k= csv_import['Wind_K']
wind_k = wind_k.values
pres_k = csv_import['Pressure_K']
pres_k = pres_k.values

print(wind_i)
print(pres_i)

print(wind_k)
print(pres_k)

#'''
#2. Calculate RMSE, correlation and p-value.
# Calculate correlation, trend and RMSE
# Variable 1 (Air Temperature)
stats = scipy.stats.spearmanr(wind_i,wind_k, axis=0, nan_policy='propagate')
cor1  = str(stats[0])
pval1 = stats[1]
cor1  = cor1[0:5]
z1    = np.polyfit(wind_i,wind_k, 1)
p1    = np.poly1d(z1)
rmse1 = np.sqrt(np.mean((wind_i-wind_k)**2))
rmse1 = str(rmse1)
rmse1 = rmse1[0:4]

# Variable 2.
# Atmospheric Pressure.
stats = scipy.stats.spearmanr(pres_i,pres_k, axis=0, nan_policy='propagate')
cor2  = str(stats[0])
pval2 = stats[1]
cor2  = cor2[0:5]
z2    = np.polyfit(pres_i,pres_k, 1)
p2    = np.poly1d(z2)

rmse2 = np.sqrt(np.mean((pres_i-pres_k)**2))
rmse2 = str(rmse2)
rmse2 = rmse2[0:4]

#4. Create variable to store X-Axis data.
# Rearrange time.
idx   = pd.date_range('2020-04-12 12:00', '2020-04-19 18:45', freq = '1H')
hours = mdates.HourLocator(interval = 6)
h_fmt = mdates.DateFormatter('%d/%M %Hh')

#3. Plot figure and set axis/data resources.
# Create figures subplots.
axs = plt.subplots(4, 1, constrained_layout=True)
fig = plt.figure()

# Y-axis range for each plots.
y_major_ticks  = np.arange(15, 32, 3)
y_major_ticks2 = np.arange(938, 953.1, 3)
y_major_ticks3 = np.arange(0, 1, 2)
y_major_ticks4 = np.arange(-4, 5, 0.5)

# Plot the figures.
# Plot 1.
ax1 = plt.subplot2grid((3,3), (0,0),rowspan=1,colspan=2)
ax1.plot(idx,wind_i,linestyle='-',label='Homemade Weather Station', marker='',linewidth=2,color='C0')  
ax1.plot(idx,wind_k,linestyle='-',label='INMET',marker='',linewidth=2,color='crimson') 
plt.fill_between(idx, wind_i, wind_k,alpha=0.3,color='gray')
ax1.legend(loc='upper left',fontsize=10)
ax1.xaxis.set_minor_locator(AutoMinorLocator())
ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(which='both',  width=2)
ax1.tick_params(which='major', length=7)
ax1.tick_params(which='minor', length=4)
ax1.set_yticks(y_major_ticks)
ax1.set_ylabel('Air Temperature \n [°C]',fontsize=12, labelpad=10)
ax1.set_title('12/04/2020 12:00 a 19/04/2020 18:45')
for label in ax1.get_xmajorticklabels():
    label.set_rotation(30)
    label.set_horizontalalignment("right")

# Plot 2.
ax2 = plt.subplot2grid((3,3), (1,0), rowspan=1,colspan=2) 
ax2.plot(idx,pres_i,label='Homemade Weather Station',linestyle='-',marker='',linewidth=2,color='C0',alpha=1)    
ax2.plot(idx,pres_k,label='INMET',linestyle='-',marker='',linewidth=2,color='crimson')
plt.fill_between(idx, pres_i, pres_k,alpha=0.3,color='gray')
ax2.legend(loc='upper left',fontsize=10)
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax2.yaxis.set_minor_locator(AutoMinorLocator())
ax2.tick_params(which='both',  width=2)
ax2.tick_params(which='major', length=7)
ax2.tick_params(which='minor', length=4)
ax2.set_yticks(y_major_ticks2)
ax2.set_ylabel('Atmospheric Pressure \n [hPa]',fontsize=12, labelpad=10)
for label in ax2.get_xmajorticklabels():
    label.set_rotation(30)
    label.set_horizontalalignment("right")

# Plot 3.
ax3 = plt.subplot2grid((3,3), (2,0)) 
ax3.scatter(wind_i,wind_k,color='C0')   
ax3.plot(wind_i,p1(wind_i),linestyle='-',color='crimson',linewidth=2)
ax3.xaxis.set_minor_locator(AutoMinorLocator())
ax3.yaxis.set_minor_locator(AutoMinorLocator())
ax3.tick_params(which='both',  width=2)
ax3.tick_params(which='major', length=7)
ax3.tick_params(which='minor', length=4)
ax3.set_yticks(y_major_ticks)
ax3.set_xticks(y_major_ticks)
ax3.set_ylabel('INMET Air Temperature \n  [°C]',fontsize=12, labelpad=10)
ax3.set_xlabel('Homemade Weather Station Air Temperature \n  [°C]',fontsize=12, labelpad=10)
ax3.text(25.6,15.2,'$r_{s}$ '+cor1,color='k',fontsize=12)
if pval1 < 0.001:
    ax3.text(25.8,16.2,'p < 0.001',color='k',fontsize=12)
else:
    ax3.text(25.8,16.2,'p > 0.001',color='k',fontsize=12)  
ax3.text(24.8,17.2,'RMSE: '+rmse1,color='k',fontsize=12)

# plot 4.
ax4 = plt.subplot2grid((3,3), (2,1),)     
ax4.scatter(pres_i,pres_k,color='C0')  
ax4.plot(pres_i,p2(pres_i),linestyle='-',color='crimson',linewidth=2)
ax4.xaxis.set_minor_locator(AutoMinorLocator())
ax4.yaxis.set_minor_locator(AutoMinorLocator())
ax4.tick_params(which='both',  width=2)
ax4.tick_params(which='major', length=7)
ax4.tick_params(which='minor', length=4)
ax4.set_yticks(y_major_ticks2)
ax4.set_xticks(y_major_ticks2)
ax4.set_ylabel('INMET Atmospheric Pressure \n [hPa]',fontsize=12, labelpad=10)
ax4.set_xlabel('Homemade Weather Station Pressure \n[hPa]',fontsize=12, labelpad=10)
ax4.text(949.2,937.2,'$r_{s}$ '+cor2,color='k',fontsize=12)
if pval2 < 0.001:
    ax4.text(949.5,938.2,'p < 0.001',color='k',fontsize=12)
else:
    ax4.text(949.5,938.2,'p > 0.001',color='k',fontsize=12)
ax4.text(948.5,939.2,'RMSE: '+rmse2,color='k',fontsize=12)


# Set final fig resources and then save figure.
fig.set_size_inches(13, 13)
fig.tight_layout()
fig.subplots_adjust(hspace = 0.3)
fig.savefig('linear_reg_time_series.png',transparent=False, bbox_inches = 'tight', pad_inches=0, dpi=250)

#'''

FileNotFoundError: [Errno 2] No such file or directory: 'G:/WRF_Schemes/Kennneth/Idai_wnsp_pres.csv'